# Diagnostics Paper Code

Code for making the figures and results tables in "Diagnostic tests for nested sampling calculations" ([Higson et al., 2018](https://arxiv.org/abs/1804.06406)). See the paper for a detailed explanation information about the plots and tables produced.

Requirements:
* Nested sampling runs saved in 'chains' - these can be generated with `generate_data.py`;
* `nestcheck`;
* `getdist` (<https://github.com/cmbant/getdist>).

Optional:
* `texunc` (<https://github.com/ejhigson/texunc>) can be used for automatically printing results tables in LaTeX format.

Figure 1 in the paper was produced with `tikz` and is not included. Output plots will be saved to `plots`.

# Set up for making plots

### Imports and settings

In [ ]:
import functools
import copy
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import nestcheck.ns_run_utils
import nestcheck.plots
import nestcheck.data_processing
import nestcheck.diagnostics_tables
import nestcheck.estimators as e
%matplotlib inline
np.random.seed(0)
pd.set_option('display.width', 200)

# Output settings
# ---------------
# Set these to match the latex
textwidth = 6.39767 * 0.99  # make 1% smaller to ensure everything fits
matplotlib.rc('text', usetex=True)
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern Roman'], 'size': 10})

# Define useful global variables
# ------------------------------
likelihood_list = ['Gaussian', 'Gaussian shell', 'Rastrigin', 'Rosenbrock']
estimator_list = [e.logz,
                  e.evidence,
                  e.param_mean,
                  functools.partial(e.param_mean, param_ind=1),
                  e.param_squared_mean,
                  functools.partial(e.param_cred, probability=0.5),
                  functools.partial(e.param_cred, probability=0.84),
                  e.r_mean,
                  functools.partial(e.r_cred, probability=0.5),
                  functools.partial(e.r_cred, probability=0.84)]

lims = {'Gaussian':       {r'$\theta_\hat{1}$':   [-2, 2],
                           r'$\theta_\hat{2}$':   [-2, 2],
                           r'$\theta_\hat{1}^2$': [0, 2.5],
                           r'$|\theta|$':  [0, 2.5]},
        'Gaussian shell': {r'$\theta_\hat{1}$':   [-4, 4],
                           r'$\theta_\hat{2}$':   [-4, 4],
                           r'$\theta_\hat{1}^2$': [0, 8],
                           r'$|\theta|$':  [1, 3]},
        'Rosenbrock':     {r'$\theta_\hat{1}$':   [-1, 2.5],
                           r'$\theta_\hat{2}$':   [-1, 5],
                           r'$\theta_\hat{1}^2$': [0, 4.5],
                           r'$|\theta|$':  [0, 4.5]},
        'Rastrigin':     {r'$\theta_\hat{1}$':    [-2.5, 2.5],
                           r'$\theta_\hat{2}$':   [-2.5, 2.5],
                           r'$\theta_\hat{1}^2$': [0, 5],
                           r'$|\theta|$':  [0, 3]}}
fthetas = {r'$\theta_\hat{1}$': lambda x: x[:, 0],
           r'$\theta_\hat{2}$': lambda x: x[:, 1],
           r'$\theta_\hat{1}^2$': lambda x: x[:, 0] ** 2,
           r'$|\theta|$': lambda x: np.sqrt(np.sum(x ** 2, axis=1))}
labels = [r'$\theta_\hat{1}$',
          r'$\theta_\hat{2}$',
          r'$\theta_\hat{1}^2$',
          r'$|\theta|$']

# Fig 2: Triangle Plots

In [ ]:
# Get runs for plotting
# ---------------------
# The same runs are used for Figures 2, 3, 4 and 5.
nlive = 100
nruns = 2
nrepeats = 5
plot_run_dict = {}
for likelihood_name in likelihood_list:
    file_root = get_file_root(likelihood_name, nlive, nrepeats)
    plot_run_dict[likelihood_name] = nestcheck.data_processing.batch_process_data(
        [file_root + '_' + str(i) for i in range(1, nruns + 1)])

# Settings
# --------

getdist_lims = copy.deepcopy(lims)
# override some lims
getdist_lims['Gaussian'][r'$\theta_\hat{1}$'] = [-2.1, 2.1]
getdist_lims['Gaussian'][r'$\theta_\hat{2}$'] = [-2.1, 2.1]
getdist_lims['Rosenbrock'][r'$\theta_\hat{1}$'] = [-1.5, 3.5]    

# Make the plots
# --------------
gplot = diagnostics.results_plots.getdist_plot(
    plot_run_dict[name], getdist_lims[name], width_inch=0.49 * textwidth)
gplot.export('plots/triangle_{}_{}nlive_{}nrepeats.pdf'.format(
    name.replace(' ', '_'), nlive, nrepeats)

# Fig 3: Posterior distributions with bootstrap uncertainty estimates

In [ ]:
# Get runs for plotting
# ---------------------
# The same runs are used for Figures 2, 3, 4 and 5.
nlive = 100
nruns = 2
nrepeats = 5
plot_run_dict = {}
for likelihood_name in likelihood_list:
    file_root = get_file_root(likelihood_name, nlive, nrepeats)
    plot_run_dict[likelihood_name] = nestcheck.data_processing.batch_process_data(
        [file_root + '_' + str(i) for i in range(1, nruns + 1)])

# Settings
# --------
n_simulate = 500  # 500 for paper
npoints = 200  # 200 for paper

labels = [r'$\theta_\hat{1}$',
          r'$\theta_\hat{2}$',
          r'$|\theta|$']


for name in likelihood_list:
    print(name)
    cache_root = 'bs_param_dists_' + name.replace(' ', '_') + '_' + str(nlive) + 'nlive_' + str(nrepeats) + 'nrepeats_' + str(n_simulate) + 'sim_' + str(npoints) + 'points'
    fig = nestcheck.plots.bs_param_dists(plot_run_dict[name],
                                         labels=labels,
                                         fthetas=[fthetas[lab] for lab in labels],
                                         ftheta_lims=[lims[name][lab] for lab in labels],
                                         cache='cache/' + cache_root,
                                         figsize=(textwidth, 1.2),
                                         n_simulate=n_simulate,
                                         rasterize_contours=True,
                                         nx=npoints, ny=npoints)
    # Ajust figure plot size manually for best use of latex space as
    # plt.layout_tight() does not work with the colorbars
    fig.subplots_adjust(left=0.03, right=0.96, bottom=0.34, top=0.98)
    fig.savefig('plots/' + cache_root + '.pdf', dpi=300)  # only contours are rasterised so dpi does not need to be that high

# Fig 4 and Fig 5: Parameter logX Diagram

In [ ]:
# Get runs for plotting
# ---------------------
# The same runs are used for Figures 2, 3, 4 and 5.
nlive = 100
nruns = 2
nrepeats = 5
plot_run_dict = {}
for likelihood_name in likelihood_list:
    file_root = get_file_root(likelihood_name, nlive, nrepeats)
    plot_run_dict[likelihood_name] = nestcheck.data_processing.batch_process_data(
        [file_root + '_' + str(i) for i in range(1, nruns + 1)])

# Settings
# --------
n_simulate = 500  # 500 for paper  
npoints = 100  # 100 for paper  


logx_min = {'Gaussian': -10,
            'Gaussian shell': -8,
            'Rosenbrock': -12,
            'Rastrigin': -15}
labels = [r'$\theta_\hat{1}$',
          r'$\theta_\hat{2}$',
          r'$|\theta|$']


for name in likelihood_list:
    if name in ['Gaussian', 'Gaussian shell']:
        run_list = plot_run_dict[name][:1]
    else:
        run_list = plot_run_dict[name]
    cache_root = 'param_logx_diagram_' + name.replace(' ', '_') + '_' + str(nlive) + 'nlive_' + str(nrepeats) + 'nrepeats_' + str(n_simulate) + 'sim'
    cache_root += '_' + str(npoints) + 'points'
    if len(run_list) != 1:
        cache_root += '_' + str(len(run_list)) + 'runs'
    fig = nestcheck.plots.param_logx_diagram(run_list,
                                             labels=labels,
                                             fthetas=[fthetas[lab] for lab in labels],
                                             ftheta_lims=[lims[name][lab] for lab in labels],
                                             cache='cache/' + cache_root,
                                             logx_min=logx_min[name],
                                             npoints=npoints,
                                             rasterize_contours=True,
                                             figsize=(textwidth * 0.49, 5))
    fig.subplots_adjust(left=0.19, right=0.97, bottom=0.08, top=0.995)
    fig.savefig('plots/' + cache_root + '.pdf', dpi=300)

# Figure 6 and Tables 1-4: Implementation error bar chart and results tables

### Get the true values of the estimators

In [ ]:
import dyPolyChord.likelihoods as likelihoods
import scipy.integrate

prior_scale = 10.

def integrand_z(x2, x1, pc_like):
    return np.exp(pc_like([x1, x2])[0]) / ((2 * prior_scale) ** 2)

def integrand_func_not_normed(x2, x1, pc_like, ftheta):
    return ftheta((x1, x2)) * np.exp(pc_like([x1, x2])[0]) / ((2 * prior_scale) ** 2)


tv_fthetas = {}
tv_fthetas[e.get_latex_name(e.param_mean)] = lambda x: x[0]
tv_fthetas[e.get_latex_name(functools.partial(e.param_mean, param_ind=1))] = lambda x: x[1]
tv_fthetas[e.get_latex_name(e.param_squared_mean)] = lambda x: x[0] ** 2
tv_fthetas[e.get_latex_name(functools.partial(e.param_squared_mean, param_ind=1))] = lambda x: x[1] ** 2
tv_fthetas[e.get_latex_name(e.r_mean)] = lambda x: np.sqrt(x[0] ** 2 + x[1] ** 2)

true_values_dict = {}
options = {"epsabs": 1.49e-11, "epsrel": 1.49e-11, 'limit': 5000}
for like in [likelihoods.gaussian, likelihoods.gaussian_shell, likelihoods.rastrigin, likelihoods.rosenbrock]:
    name = like.__name__.replace('_', ' ').title().replace('Shell', 'shell')
    print(name)
    z = scipy.integrate.nquad(integrand_z, ranges=[(-prior_scale, prior_scale), (-prior_scale, prior_scale)],
                              args=(like,), opts=options)
    true_values_dict[name] = {}
    true_values_dict[name][e.get_latex_name(e.logz)] = np.log(z[0])

    for ftheta_name, ftheta in tv_fthetas.items():
        val = scipy.integrate.nquad(integrand_func_not_normed, ranges=[(-prior_scale, prior_scale), (-prior_scale, prior_scale)],
                                    args=(like, ftheta), opts=options)
        true_values_dict[name][ftheta_name] = val[0] / z[0]

### Get error results data frame

In [ ]:
# Settings
# --------
nlive = 100
nrepeats = 5
n_runs = 500
n_simulate = 100


# Get data
errors_df = diagnostics.results_utils.get_results_df(
    likelihood_list, [(nlive, nrepeats)], estimator_list, n_simulate=n_simulate,
    n_runs=n_runs, summary=True, save=True, load=True, thread_pvalue=False,
    bs_stat_dist=False, true_values_dict=true_values_dict, include_rmse=True,
    include_true_values=True)

# Format_data
estimator_names_bar = [e.get_latex_name(est) for est in [e.logz,
                                                         e.param_mean,
                                                         functools.partial(e.param_mean, param_ind=1),
                                                         e.r_mean,
                                                         e.param_squared_mean]]
errors_df = errors_df.xs(nrepeats, level='nrepeats').xs(nlive, level='nlive')[estimator_names_bar]

### Plot bar chart (Fig 5)

In [ ]:
# Make bar chart
ratio_plot = errors_df.xs('implementation std frac', level='calculation type')
ratio_plot = ratio_plot.reorder_levels([1, 0]).T
fig = plt.figure(figsize=(textwidth * 0.8, 2))
ax = fig.add_subplot(111)
ratio_plot['value'].plot.bar(yerr=ratio_plot['uncertainty'], ax=ax)
# Add line showing 1/sqrt(2)
ax.axhline(2 ** (-0.5), color='black',
           linestyle='dashed', linewidth=1)
# ax = plt.gca()
ax.set_ylim([0, 1])
ax.set_ylabel('Imp St.Dev. / Values St.Dev.', labelpad=10)
ax.legend(bbox_to_anchor=(1.02, 1), title='Likelihood')
plt.xticks(rotation=0)
savename = ('plots/imp_error_test_' + str(n_runs) + 'runs_' + str(n_simulate) + 'sim_' + str(nlive) + 'nlive_'+
            str(nrepeats) + 'nrepeats.pdf')
fig.subplots_adjust(left=0.11, right=0.7, bottom=0.14, top=0.97)
fig.savefig(savename)

In [ ]:
# Make results tables
str_map = {'true values': 'Correct Result',
           'values mean': 'Mean Calculation Result',
           'values std': r'Values St.Dev.\ ',
           'values rmse': 'Values RMSE',
           'bootstrap std mean': r'Bootstrap St.Dev.\ ',
           'implementation std': r'Implementation St.Dev.\ ',
           'implementation std frac': r'Imp St.Dev. / Val St.Dev.\ ',
           r'Implementation St.Dev.\  frac': r'Imp St.Dev./Val St.Dev.\ ',
           'mathrm{log}': 'log'}
df_dict = {}
for likelihood_name in likelihood_list:
    df = errors_df.xs(likelihood_name, level='likelihood')
    label = 'tab:' + likelihood_name.lower().replace(' ', '_')
    caption = ('As in \Cref{tab:gaussian} but for calculations using the ' + likelihood_name +
               r' likelihood~\eqref{equ:' + likelihood_name.lower().replace(' ', '_') + r'}.')
    try:
        import texunc
        df = texunc.print_latex_df(
            df, min_dp=1, min_dp_no_error=4, str_map=str_map, caption=caption,
            caption_above=False, label=label, zero_dp_ints=False)
    except ImportError:
        pass
    # Also store the formatted df
    df.index = [str_map[ind] for ind in list(df.index)]
    df_dict[likelihood_name] = df
pd.concat(df_dict)

# Figures 7 and 8:  Line plots of errors vs nlive and nrepeats

### Get a data frame of results

In [ ]:
# Settings
# --------
n_runs_lp = 500
n_simulate_lp = 10
nlive_nrepeats_list = [(nl, 5) for nl in [10, 20, 50, 100, 200, 500, 1000]]
nlive_nrepeats_list += [(100, nr) for nr in [1, 2, 10, 20, 50, 100, 200, 500, 1000]]
# Run plots
results_df_in = diagnostics.results_utils.get_results_df(
    likelihood_list, nlive_nrepeats_list, estimator_list, n_simulate=n_simulate_lp,
    n_runs=n_runs_lp, summary=True, save=True, load=True, thread_pvalue=False,
    bs_stat_dist=False, true_values_dict=true_values_dict, include_rmse=True,
    include_true_values=True)

### Make line plots

In [ ]:
from matplotlib.ticker import FormatStrFormatter
linestyles = ['-', '--', ':', '-.']
x_label_map = {'nlive': '{\sc PolyChord} number of live points', 'nrepeats': '{\sc PolyChord} \\texttt{num\_repeats}'}
ests_for_line_plot = [e.logz, e.param_mean]
est_savename_map = {}
for est in ests_for_line_plot:
    est_savename_map[e.get_latex_name(est)] = est.__name__
for df_temp in [results_df_in.xs(5, level='nrepeats'), results_df_in.xs(100, level='nlive')]:
    for estimator_name in [e.get_latex_name(est) for est in ests_for_line_plot]:
        print(estimator_name)
        fig, axes = plt.subplots(nrows=len(likelihood_list), ncols=1, sharex=True, figsize=(textwidth / 2, 6))
        fig.subplots_adjust(hspace=0)
        for nlike, likelihood_name in enumerate(likelihood_list):
            ax = axes[nlike]
            for nc, calc in enumerate(['values std', 'bootstrap std mean', 'implementation std']):
                ser = df_temp.xs(likelihood_name, level='likelihood').xs(calc, level='calculation type')[estimator_name]
                ser = ser.sort_index()
                ser.xs('value', level='result type').plot.line(
                        yerr=ser.xs('uncertainty', level='result type'),
                        ax=ax, label=calc, linestyle=linestyles[nc])
            ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
            if 'nlive' in df_temp.index.names and likelihood_name == 'Gaussian' and estimator_name == e.get_latex_name(e.param_mean):
                ax.set_yticks([0, 0.05, 0.1])
            ax.set_xscale('log')
            ax.set_ylabel('St.Dev.')
            title = likelihood_name.title().replace('_', ' ').replace('Shell', 'shell') + ' ' + estimator_name
            ax.set_title(title, y=0.72)
            # make sure the labels of plots above and below each other don't clash
            ax.set_ylim([0, ax.get_yticks()[-1]])
            ax.tick_params(top=True, direction='inout')
            if nlike != 0:
                labels = ax.get_yticks().tolist()
                ax.set_yticks(labels[:-1])
            if nlike == len(likelihood_list) - 1:
                ax.set_xlabel(x_label_map[ax.get_xlabel()])
        savename = 'plots/line'
        if 'nlive' in df_temp.index.names:
            savename += '_nlive'
        elif 'nrepeats' in df_temp.index.names:
            savename += '_nrepeats'
        savename += '_' + str(n_runs_lp) + 'runs_' + str(n_simulate_lp) + 'sim_' + est_savename_map[estimator_name] + '.pdf'
        # Manually adjust saving as described in https://matplotlib.org/devdocs/api/_as_gen/matplotlib.pyplot.subplots_adjust.html
        fig.subplots_adjust(hspace=0, left=0.17, right=0.995, bottom=0.07, top=0.99)
        fig.savefig(savename)

### Make legend in seperate file

In [ ]:
fig = plt.figure(figsize=(textwidth, 0.3))
for i, label in enumerate(['result values', 'mean bootstrap estimate', 'implementation error']):
    plt.plot([0,1],[-1,-1], label=label)
plt.legend(ncol=3, loc='center')
plt.gca().set_ylim(bottom=0)
plt.axis('off')
fig.savefig('plots/line_plot_legend.pdf')

# Figures 9, 11, 12 and 13: Histograms

In [ ]:
# Settings
# --------
nlive = 1000
nrepeats = 5
n_runs = 500
n_simulate = 1000
vals_df_in = diagnostics.results_utils.get_results_df(
    likelihood_list, [(nlive, nrepeats)], estimator_list, n_simulate=nsimulate, n_runs=n_runs,
    summary=False, save=True, load=True, thread_pvalue=True, bs_stat_dist=True)

In [ ]:
vals_df_in

In [ ]:
xlims = {'thread ks pvalue': [0,1],
         'thread ks distance': [0,0.3],
         'thread earth mover distance': [0, 0.2],
         'thread energy distance': [0, 0.4],
         'bootstrap ks pvalue': [0,1],
         'bootstrap ks distance': [0,1],
         'bootstrap earth mover distance': [0, 0.25],
         'bootstrap energy distance': [0, 0.8]}

for i, est in enumerate([e.param_mean, functools.partial(e.param_mean, param_ind=1)]):
    for calc in ['thread ks pvalue', 'bootstrap ks distance', 'bootstrap earth mover distance', 'bootstrap energy distance']:
        print(calc)
        fig = diagnostics.results_plots.hist_plot(vals_df_in, calc, e.get_latex_name(est), xlims[calc], nbin=50, figsize=(textwidth, 1.4))
        savename = 'plots/hist_' + calc.replace(' ', '_') + '_theta' + str(i + 1) + '_' + str(n_runs) + 'runs_' + str(n_simulate) + 'sim_' + str(nlive) + 'nlive_'+ str(nrepeats) + 'nrepeats'
        savename = savename.replace('.', '_') + '.pdf'
        fig.subplots_adjust(left=0.096, right=0.985, bottom=0.29, top=0.98)
        fig.savefig(savename)

# Figure 10: 1d KDE distributions of bootstrap sampling error estimates

In [ ]:
# Settings
# --------
nlive = 1000
nruns = 5
nrepeats = 5
n_simulate = 1000

# Get runs
kde_run_dict = {}
for likelihood_name in likelihood_list:
    file_root = get_file_root(likelihood_name, nlive, nrepeats)
    kde_run_dict[likelihood_name] = nestcheck.data_processing.batch_process_data(
        [file_root + '_' + str(i) for i in range(1, nruns + 1)])

estimator_list_1dkde = [e.logz,
                        e.param_mean,
                        functools.partial(e.param_mean, param_ind=1)]
estimator_names_1dkde = [e.get_latex_name(est) for est in estimator_list_1dkde]
    
bs_dict = {}
for likelihood_name in likelihood_list:
    bs_dict[likelihood_name] = nestcheck.diagnostics_tables.bs_values_df(
        kde_run_dict[likelihood_name], estimator_list_1dkde, estimator_names_1dkde, n_simulate)

In [ ]:
for likelihood_name in likelihood_list:
    print(likelihood_name)
    bs_df = bs_dict[likelihood_name].iloc[[2, 3]]
    fig = nestcheck.plots.kde_plot_df(
        bs_df, figsize=(textwidth * 0.5, 1.2), num_xticks=2)
    fig.subplots_adjust(left=0.03, right=0.97, bottom=0.35, top=0.99)
    fig.savefig('plots/1dkde_' + likelihood_name.replace(' ', '_') + '_' + str(nlive) + 'nlive_' + str(nrepeats) + 'nrepeats_' + str(n_simulate) + 'sim.pdf')